### 카테고리별 지원 파싱 (노령자지원)
노령자지원 (216~234페이지) 파싱

In [1]:
import os
import json
import time
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import JsonOutputParser
from pydantic import BaseModel, Field
import fitz

load_dotenv()

True

In [2]:
class TableData(BaseModel):
    columns: list = Field(description="표의 열 제목들")
    rows: list = Field(description="표의 행 데이터들")

class WelfareProgram(BaseModel):
    title: str = Field(description="복지 프로그램의 정확한 지원명")
    target: str = Field(description="지원 대상자")
    content: str = Field(description="지원 내용")
    method: str = Field(description="신청 방법")
    inquiry: str = Field(description="문의처")
    tables: list[TableData] = Field(description="표 데이터 구조")

In [3]:
def extract_page_text(pdf_path, page_num):
    doc = fitz.open(pdf_path)
    page = doc.load_page(page_num - 1)
    text = page.get_text("text")
    doc.close()
    return text

In [4]:
def parse_elderly_support(text, page_num):
    llm = ChatOpenAI(
        model="gpt-3.5-turbo",
        temperature=0,
        request_timeout=60,
        max_retries=3
    )
    
    parser = JsonOutputParser(pydantic_object=WelfareProgram)
    
    prompt = ChatPromptTemplate.from_messages([
        ("system", """노령자지원 문서 전용 파서입니다.

검증 로직:
1. "대상", "내용", "방법", "문의" 4개 필수 키워드 존재 확인
2. 키워드 중 하나라도 없으면 null 반환

제도명 추출 규칙:
- "나에게 힘이 되는 복지서비스"는 페이지 헤더이므로 제외
- "노령자 지원"은 카테고리명이므로 제외
- 실제 제도명만 추출 (예: "기초연금", "노인장기요양보험", "노인일자리사업")

표 처리 규칙:
1. content: 표 내용을 자연어로 요약하여 포함
2. tables: 표가 있으면 구조화된 데이터로 저장
   - columns: 실제 표의 열 제목을 정확히 추출 (예: ["연령", "지급액", "비고"])
   - rows: 실제 표의 행 데이터를 정확히 추출 (예: [["65세 이상", "월 34만원", "기초연금"]])
   - 표의 실제 구조를 그대로 반영하여 저장
3. 표가 없으면 tables는 빈 배열

추출 최적화:
- 제도명: 실제 노령자지원제도명만
- 대상: 연령기준, 소득기준, 건강상태 등 구체적 조건
- 내용: 지원금액, 지원기간, 지원내용 (표 데이터 요약 포함)
- 방법: 신청기관, 온라인/오프라인 구분, 필요서류
- 문의: 콜센터, 웹사이트, 담당기관
- tables: 표가 있으면 실제 컬럼명과 데이터로 구조화하여 저장

{format_instructions}"""),

        ("human", """노령자지원 문서를 분석합니다.

1단계: 필수 키워드 확인
- "대상", "내용", "방법", "문의" 4개 키워드 모두 존재하는지 확인
- 하나라도 없으면 null 반환

2단계: 제도명 추출
- "나에게 힘이 되는 복지서비스" 제외
- "노령자 지원" 제외
- 실제 노령자지원제도명만 추출

3단계: 정보 추출
- content: 표 내용을 자연어로 요약하여 포함
- tables: 표가 있으면 실제 컬럼명과 행 데이터를 정확히 추출하여 저장
- 각 키워드 바로 다음 내용을 정확히 추출

텍스트:
{text}
""")
    ])
    
    chain = prompt | llm | parser
    
    for attempt in range(3):
        try:
            result = chain.invoke({
                "text": text,
                "format_instructions": parser.get_format_instructions()
            })
            
            if result is None:
                print(f"페이지 {page_num}: 필수 키워드 부족으로 건너뛰기")
                return None
            
            result["page"] = page_num
            result["category"] = "노령자지원"
            
            return result
        except Exception as e:
            print(f"파싱 시도 {attempt + 1}/3 실패 (페이지 {page_num}): {e}")
            if attempt < 2:
                time.sleep(2)
            else:
                return None

In [5]:
# 실행
pdf_path = "../../data/pdfs/[통합]+2025+나에게+힘이+되는+복지서비스.pdf"
target_pages = list(range(216, 235))  # 노령자지원 섹션 (216~234페이지)
elderly_programs = []

for i, page_num in enumerate(target_pages):
    print(f"페이지 {page_num} 파싱 중... ({i+1}/{len(target_pages)})")
    
    page_text = extract_page_text(pdf_path, page_num)
    program = parse_elderly_support(page_text, page_num)
    
    if program:
        elderly_programs.append(program)
        table_count = len(program.get('tables', []))
        print(f" {program['title']} 추출 완료 (표 {table_count}개)")
    else:
        print(f"페이지 {page_num} 건너뛰기")
    
    if i < len(target_pages) - 1:
        time.sleep(1)

페이지 216 파싱 중... (1/19)
 기초연금제도 추출 완료 (표 1개)
페이지 217 파싱 중... (2/19)
 노후준비서비스 추출 완료 (표 0개)
페이지 218 파싱 중... (3/19)
 주택연금제도 추출 완료 (표 0개)
페이지 219 파싱 중... (4/19)
 노후긴급자금 대부 사업 추출 완료 (표 0개)
페이지 220 파싱 중... (5/19)
 노인 일자리 및 사회활동 지원 추출 완료 (표 0개)
페이지 221 파싱 중... (6/19)
 노인 일자리 및 사회활동 지원 추출 완료 (표 0개)
페이지 222 파싱 중... (7/19)
 디지털배움터 추출 완료 (표 0개)
페이지 223 파싱 중... (8/19)
 취약지역 어르신 문화누림 추출 완료 (표 0개)
페이지 224 파싱 중... (9/19)
 치매검진 지원 추출 완료 (표 1개)
페이지 225 파싱 중... (10/19)
 치매치료 관리비 지원사업 추출 완료 (표 0개)
페이지 226 파싱 중... (11/19)
 노인 안검진 및 개안수술 추출 완료 (표 0개)
페이지 227 파싱 중... (12/19)
 노인 틀니 지원 추출 완료 (표 1개)
페이지 228 파싱 중... (13/19)
 노인 치과 임플란트 지원 추출 완료 (표 1개)
페이지 229 파싱 중... (14/19)
 노인 무릎인공관절 수술 지원 추출 완료 (표 0개)
페이지 230 파싱 중... (15/19)
 어르신 국가예방접종 지원 사업 추출 완료 (표 0개)
페이지 231 파싱 중... (16/19)
 노인맞춤돌봄서비스 추출 완료 (표 1개)
페이지 232 파싱 중... (17/19)
 응급안전 안심서비스 추출 완료 (표 0개)
페이지 233 파싱 중... (18/19)
 노인학대예방 및 지원 프로그램 추출 완료 (표 0개)
페이지 234 파싱 중... (19/19)
 어르신을 위한 요금감면 제도 추출 완료 (표 1개)


In [6]:
# 결과 저장
os.makedirs("outputs", exist_ok=True)
output_path = "../../data/outputs/노령자지원_2025.json"

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(elderly_programs, f, ensure_ascii=False, indent=2)

print(f"\n파싱 완료: {len(elderly_programs)}개 프로그램")
print(f"저장 위치: {output_path}")


파싱 완료: 19개 프로그램
저장 위치: ../../data/outputs/노령자지원_2025.json


In [7]:

for program in elderly_programs:
    table_count = len(program.get('tables', []))
    print(f"\n{program['title']} (페이지 {program['page']}, 표 {table_count}개)")
    print(f"대상: {program['target'][:80]}...")
    print(f"내용: {program['content'][:80]}...")
    if table_count > 0:
        print(f"표: {program['tables'][0]['columns'][:3]}...")


기초연금제도 (페이지 216, 표 1개)
대상: 65세 이상이며, 소득인정액이 선정기준액(소득하위 70%) 이하인 어르신...
내용: 월 최대 34만 2,510원까지 지원. 선정기준액: 소득하위 70%. 단독가구 228만 원, 부부가구 364.8만 원. 월 최대 34만 2,51...
표: ['구분', '선정기준액', '월 최대 지원 금액']...

노후준비서비스 (페이지 217, 표 0개)
대상: 국민연금 가입대상 및 수급권자를 포함한 전 국민...
내용: 노후준비서비스는 재무, 건강, 여가, 대인관계 분야의 표준지표로 구성된 '종합진단지'를 작성하여 노후준비 상태를 진단하고, 취약점을 보완하기 위...

주택연금제도 (페이지 218, 표 0개)
대상: 만 55세 이상 가입자 (보유 주택 공시 가격이 12억 원 이하)...
내용: 주택을 담보로 맡기고 내 집에 살면서 매월 연금을 수령하는 제도. 예를 들어, 3억 원 집값의 55세 가입자는 매월 44만 3,730원, 70세...

노후긴급자금 대부 사업 (페이지 219, 표 0개)
대상: 만 60세 이상의 국민연금 수급자...
내용: 노후긴급자금 대부 사업은 만 60세 이상의 국민연금 수급자를 대상으로 전･월세보증금, 의료비, 배우자 장제비, 재해복구비 용도의 긴급 자금을 저...

노인 일자리 및 사회활동 지원 (페이지 220, 표 0개)
대상: 65세 이상 기초연금 수급자, 60세 이상 노인, 60세 이상 시장형사업단, 60세 이상 취업알선형, 60세 이상 시니어인턴십, 고령자친화기업...
내용: 다양한 일자리 및 사회활동 지원 프로그램이 제공됩니다. 사회활동 및 공익활동 참여를 통해 지역사회 공익 증진 프로그램에 참여하고, 사회서비스형 ...

노인 일자리 및 사회활동 지원 (페이지 221, 표 0개)
대상: 노인 일자리 및 사회활동을 필요로 하는 노인...
내용: 노인 일자리 및 사회활동 지원 사업은 주민센터(행정복지센터), 노인일자리 수행기관(시니어클럽, 노인복지관 등)에서 신청할 수 있습니다. 노인